In [1]:
import numpy as np
import pandas as pd
from scipy.special import boxcox, inv_boxcox
from scipy.stats import boxcox_normmax
import matplotlib.pyplot as plt
from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS
from neuralforecast.losses.pytorch import MAE, DistributionLoss
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
df_up = pd.read_csv("../data/5_yr_data/UP5_years.csv")
df_up['datetime'] = pd.to_datetime(df_up['date'])
df_up.drop(columns=["Unnamed: 0"], axis=1, inplace=True)
df_up.sort_values(by="datetime", ascending=True, inplace=True)
df_up.head()

,district_name,market_name,commodity,variety,grade,min_rs_quintal,max_rs_quintal,modal_rs_quintal,date,year,month,day_of_month,datetime
714742,Bijnor,Bijnaur,Onion,Red,FAQ,2950.0,3040.0,3000.0,01 Jan 2018,2018,Jan,1,2018-01-01
433767,Mau(Maunathbhanjan),Kopaganj,Wheat,Dara,FAQ,1525.0,1625.0,1575.0,01 Jan 2018,2018,Jan,1,2018-01-01
439485,Gorakhpur,Gorakhpur,Wheat,Dara,FAQ,1560.0,1590.0,1575.0,01 Jan 2018,2018,Jan,1,2018-01-01
83163,Shahjahanpur,Tilhar,Potato,Potato,FAQ,490.0,510.0,500.0,01 Jan 2018,2018,Jan,1,2018-01-01
730321,Bulandshahar,Divai,Onion,Red,FAQ,2800.0,3000.0,2900.0,01 Jan 2018,2018,Jan,1,2018-01-01


In [ ]:
TRAIN_LEN = int(0.8 * len(df_up))
up_train, up_test = (df_up[:TRAIN_LEN],df_up[TRAIN_LEN:])
up_train.set_index('datetime', inplace=True)
up_train.sort_index(inplace=True)
up_test.set_index('datetime', inplace=True)
up_test.sort_index(inplace=True)

In [ ]:
commodity = "Rice"
df_train_commodity = up_train[up_train['commodity'] == commodity]
df_train_commodity_dt = df_train_commodity.groupby("datetime").agg({"modal_rs_quintal":"mean"})
df_train_commodity_dt

,modal_rs_quintal
datetime,
2018-01-01,2244.840426
2018-01-02,2261.681818
2018-01-03,2235.193548
2018-01-04,2258.073684
2018-01-05,2291.752941
...,...
2023-04-29,2648.738739
2023-04-30,2665.293103
2023-05-01,2614.902913


In [ ]:
df_test_commodity = up_test[up_test['commodity'] == commodity]
df_test_commodity_dt = df_test_commodity.groupby("datetime").agg({"modal_rs_quintal":"mean"})
df_test_commodity_dt.head()

,modal_rs_quintal
datetime,
2023-05-03,2581.857143
2023-05-04,2681.118421
2023-05-05,2662.755102
2023-05-06,2594.227273
2023-05-07,2559.982456


In [ ]:
df_train_commodity_dt.reset_index(inplace=True)
df_train_commodity_dt['unique_id'] = commodity
df_train_commodity_dt.rename(columns={"datetime" : "ds", "modal_rs_quintal" : "y"}, inplace=True)
df_train_commodity_dt.head()

,ds,y,unique_id
0,2018-01-01,2244.840426,Rice
1,2018-01-02,2261.681818,Rice
2,2018-01-03,2235.193548,Rice
3,2018-01-04,2258.073684,Rice
4,2018-01-05,2291.752941,Rice


In [ ]:
df_test_commodity_dt.reset_index(inplace=True)
df_test_commodity_dt['unique_id'] = commodity
df_test_commodity_dt.rename(columns={"datetime" : "ds", "modal_rs_quintal" : "y"}, inplace=True)
df_test_commodity_dt.head()

,ds,y,unique_id
0,2023-05-03,2581.857143,Rice
1,2023-05-04,2681.118421,Rice
2,2023-05-05,2662.755102,Rice
3,2023-05-06,2594.227273,Rice
4,2023-05-07,2559.982456,Rice


In [ ]:
df_train_commodity_dt.shape

(1949, 3)

In [ ]:
df_test_commodity_dt.shape

(488, 3)

In [ ]:
# Required imports
import pandas as pd
import numpy as np
from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS, NHITS, DeepAR, TFT, LSTM, RNN, GRU
from neuralforecast.losses.pytorch import DistributionLoss, MAE, MSE, MAPE, SMAPE
import torch

from darts import TimeSeries
from darts.models import (
    NBEATSModel,
    NHiTSModel,
    BlockRNNModel,
    TCNModel,
    TiDEModel,
    TransformerModel,
    RandomForest,
    LightGBMModel,
    XGBModel,
    Prophet,
)


def create_nixtla_models(input_size=120, output_size=368):
    """
    Create a collection of Nixtla models with correct parameters
    """
    # Common parameters
    common_params = {
        "input_size": input_size,
        "h": output_size,
        "max_steps": 100,
        "val_check_steps": 16,
        "early_stop_patience_steps": 4,
    }

    # N-BEATS model
    nbeats = NBEATS(
        **common_params,
        loss=DistributionLoss(distribution="Normal", level=[80, 90]),
        stack_types=["trend", "seasonality"],
        num_blocks=[3, 3],
        num_layers=[4, 4],
        layer_widths=[256, 2048],
        expansion_coefficient_lengths=[5, 7],
        trend_polynomial_degree=2,
    )

    # N-HiTS model
    nhits = NHITS(
        **common_params,
        loss=DistributionLoss(distribution="Normal", level=[80, 90]),
        num_stacks=3,  # Default is 3
        hidden_size=128,  # Units per hidden layer
        n_freq_downsample=[168, 24, 1],  # Pooling factor per stack
        pooling_kernel_sizes=[168, 24, 1],
        interpretation=False,
        activation="ReLU",
    )

    # DeepAR model
    deepar = DeepAR(
        **common_params,
        loss=DistributionLoss(distribution="StudentT", level=[80, 90]),
        hidden_size=128,
        rnn_layers=2,
        dropout=0.1,
        cell_type="LSTM",
    )

    # Temporal Fusion Transformer
    tft = TFT(
        **common_params,
        loss=DistributionLoss(distribution="Normal", level=[80, 90]),
        hidden_size=128,  # Hidden state size
        lstm_hidden_size=64,  # Size of LSTM hidden states
        num_attention_heads=4,  # Number of attention heads
        dropout=0.1,  # Dropout rate
        hidden_continuous_size=64,  # Size for processing continuous variables
    )

    # LSTM model
    lstm = LSTM(
        **common_params,
        loss=MSE(),
        hidden_size=128,
        num_layers=2,
        dropout=0.1,
        batch_normalization=True,
    )

    # RNN model
    rnn = RNN(
        **common_params,
        loss=MAE(),
        hidden_size=128,
        num_layers=2,
        dropout=0.1,
        cell_type="GRU",
    )

    # GRU model
    gru = GRU(**common_params, loss=SMAPE(), hidden_size=128, num_layers=2, dropout=0.1)

    # Create NeuralForecast object with all models
    fcst = NeuralForecast(models=[nbeats, nhits, deepar, tft, lstm, rnn, gru], freq="D")

    return fcst


def create_darts_models(input_chunk_length=120, output_chunk_length=30, n_epochs=100):
    """
    Create a collection of Darts models with correct parameters
    """
    # Common parameters for neural networks
    nn_params = {
        "input_chunk_length": input_chunk_length,
        "output_chunk_length": output_chunk_length,
        "n_epochs": n_epochs,
        "batch_size": 32,
        "force_reset": True,
    }

    models = {
        # Neural network based models
        "nbeats": NBEATSModel(
            **nn_params,
            generic_architecture=False,
            num_stacks=2,
            num_blocks=3,
            num_layers=4,
            layer_widths=256,
            expansion_coefficient_dim=5,
            trend_polynomial_degree=2,
        ),
        "nhits": NHiTSModel(
            **nn_params,
            num_stacks=3,
            num_blocks=1,
            num_layers=2,
            layer_widths=512,
            pooling_kernel_sizes=None,
            n_freq_downsample=None,
            dropout=0.1,
            activation="ReLU",
            MaxPool1d=True,
        ),
        "block_rnn": BlockRNNModel(
            **nn_params,
            model="LSTM",
            hidden_dim=128,
            n_rnn_layers=2,
            dropout=0.1,
        ),
        "tcn": TCNModel(
            **nn_params,
            num_filters=64,
            kernel_size=3,
            dilation_base=2,
            dropout=0.1,
            weight_norm=True,
        ),
        "tide": TiDEModel(
            **nn_params,
            num_encoder_layers=2,
            num_decoder_layers=2,
            temporal_width_past=24,
            temporal_width_future=12,
            temporal_decoder_hidden=32,
        ),
        "transformer": TransformerModel(
            **nn_params,
            d_model=64,
            nhead=4,
            num_encoder_layers=3,
            num_decoder_layers=3,
            dim_feedforward=256,
            dropout=0.1,
            activation="gelu",
        ),
        # Traditional ML models
        "random_forest": RandomForest(
            lags=input_chunk_length,
            n_estimators=100,
            max_depth=None,
            min_samples_split=2,
        ),
        
        "xgboost": XGBModel(
            lags=input_chunk_length, n_estimators=100, max_depth=6, learning_rate=0.1
        ),
        
    }

    return models


def train_and_forecast(df_train, df_test, use_nixtla=True):
    """
    Train models and generate forecasts using either Nixtla or Darts
    """
    if use_nixtla:
        # Nixtla workflow
        fcst = create_nixtla_models()

        # Ensure df_train has the required columns
        if "unique_id" not in df_train.columns:
            df_train["unique_id"] = "series0"
        if "ds" not in df_train.columns:
            df_train = df_train.rename(columns={"date": "ds"})
        if "y" not in df_train.columns:
            df_train = df_train.rename(columns={"value": "y"})

        # Similarly for test data
        if "unique_id" not in df_test.columns:
            df_test["unique_id"] = "series0"
        if "ds" not in df_test.columns:
            df_test = df_test.rename(columns={"date": "ds"})
        if "y" not in df_test.columns:
            df_test = df_test.rename(columns={"value": "y"})

        fcst.fit(df=df_train, val_size=488)
        forecasts = fcst.predict(futr_df=df_test)
        return forecasts
    else:
        # Darts workflow
        # Convert pandas DataFrame to Darts TimeSeries
        series = TimeSeries.from_dataframe(df_train, "ds", "y")

        # Create and train models
        models = create_darts_models()
        forecasts = {}

        for name, model in models.items():
            print(f"Training {name} model...")
            model.fit(series)
            forecast = model.predict(len(df_test))
            forecasts[name] = forecast

        return forecasts


# Example usage:
"""
# For Nixtla models
nixtla_forecasts = train_and_forecast(df_train_commodity_dt, df_test_commodity_dt, use_nixtla=True)

# For Darts models
darts_forecasts = train_and_forecast(df_train_commodity_dt, df_test_commodity_dt, use_nixtla=False)
"""

'\n# For Nixtla models\nnixtla_forecasts = train_and_forecast(df_train_commodity_dt, df_test_commodity_dt, use_nixtla=True)\n\n# For Darts models\ndarts_forecasts = train_and_forecast(df_train_commodity_dt, df_test_commodity_dt, use_nixtla=False)\n'

In [ ]:
y_train = df_train_commodity_dt['y']


,ds,y,unique_id
0,2018-01-01,2244.840426,Rice
1,2018-01-02,2261.681818,Rice
2,2018-01-03,2235.193548,Rice
3,2018-01-04,2258.073684,Rice
4,2018-01-05,2291.752941,Rice
...,...,...,...
1944,2023-04-29,2648.738739,Rice
1945,2023-04-30,2665.293103,Rice
1946,2023-05-01,2614.902913,Rice
1947,2023-05-02,2653.594340,Rice


In [ ]:
nixtla_forecasts = train_and_forecast(
    df_train_commodity_dt, df_test_commodity_dt, use_nixtla=False
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | criterion       | MSELoss          | 0      | train
1 | train_criterion | MSELoss          | 0      | train
2 | val_criterion   | MSELoss          | 0      | train
3 | train_metrics   | MetricCollection | 0      | train
4 | val_metrics     | MetricCollection | 0      | train
5 | stacks          | ModuleList       | 511 K  | train
-------------------------------------------------------------
465 K     Trainable params
46.2 K    Non-trainable params
511 K     Total params
2.048     Total estimated model params size (MB)
32        Modules in train mode
0         Modules in eval mode


Training nbeats model...


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

...

Training transformer model...


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

Training random_forest model...
Training xgboost model...


In [ ]:
nixtla_forecasts

{'nbeats': <TimeSeries (DataArray) (ds: 488, component: 1, sample: 1)> Size: 4kB
 array([[[2639.33696788]],
 
        [[2615.77879673]],
 
        [[2625.00163047]],
 
        [[2640.62315947]],
 
        [[2627.40220068]],
 
        [[2595.34941697]],
 
        [[2601.16252655]],
 
        [[2638.49261573]],
 
        [[2608.34834722]],
 
        [[2618.15027147]],
 
 ...
 
        [[2596.88305078]],
 
        [[2591.0446558 ]],
 
        [[2582.96959298]],
 
        [[2584.52410646]],
 
        [[2591.94867862]],
 
        [[2587.26991158]],
 
        [[2590.07644368]],
 
        [[2600.14847853]],
 
        [[2589.23266195]],
 
        [[2588.09429026]]])
 Coordinates:
   * ds         (ds) datetime64[ns] 4kB 2023-05-04 2023-05-05 ... 2024-09-02
   * component  (component) object 8B 'y'
 Dimensions without coordinates: sample
 Attributes:
     static_covariates:  None
     hierarchy:          None,
 'nhits': <TimeSeries (DataArray) (ds: 488, component: 1, sample: 1)> Size: 4kB
 array